# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [3]:
# Load the libraries as required.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, PowerTransformer, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [5]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()
fires_dt.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [6]:
X = fires_dt.drop(columns=['area'])
y = fires_dt['area']

# Display the shape of X and y to confirm
print("Features (X) shape:", X.shape)
print("Target (y) shape:", y.shape)

Features (X) shape: (517, 12)
Target (y) shape: (517,)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [7]:
numerical_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Preprocessor 1: RobustScaler + One-Hot Encoding (No Non-Linear Transform)
preproc1 = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
preproc1

ColumnTransformer(transformers=[('num', RobustScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [8]:
# Preprocessor 2: RobustScaler + Non-Linear Transform + One-Hot Encoding
preproc2 = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numerical_features),
        ('num_non_linear', PowerTransformer(), ['ffmc', 'isi']),  # Example: Apply PowerTransform to FFMC and ISI
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
preproc2

ColumnTransformer(transformers=[('num', RobustScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('num_non_linear', PowerTransformer(),
                                 ['ffmc', 'isi']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [9]:
# Pipeline A = preproc1 + baseline

# Define Pipeline A: preproc1 + Decision Tree Regressor (Baseline)
pipeline_a_dtr = Pipeline(steps=[
    ('preprocessing', preproc1),  # Preprocessing pipeline
    ('regressor', DecisionTreeRegressor(random_state=42))  # Decision Tree Regressor
])

# Evaluate Pipeline A using cross_validate with error metrics
print("Evaluating Pipeline A (preproc1 + Decision Tree Regressor)...")
cv_results = cross_validate(
    pipeline_a_dtr,
    X,
    y,
    cv=5,
    scoring=['neg_mean_squared_error', 'neg_mean_absolute_error']  # Only error metrics
)

# Display results for each fold
print("\nCross-Validation Results:")
for i in range(5):
    print(f"Fold {i+1}: "
          f"MSE = {-cv_results['test_neg_mean_squared_error'][i]:.2f}, "
          f"MAE = {-cv_results['test_neg_mean_absolute_error'][i]:.2f}")

# Display mean metrics
mean_mse = -cv_results['test_neg_mean_squared_error'].mean()
mean_mae = -cv_results['test_neg_mean_absolute_error'].mean()

print(f"\nMean MSE: {mean_mse:.2f}")
print(f"Mean MAE: {mean_mae:.2f}")


Evaluating Pipeline A (preproc1 + Decision Tree Regressor)...

Cross-Validation Results:
Fold 1: MSE = 3733.03, MAE = 27.96
Fold 2: MSE = 2017.10, MAE = 19.01
Fold 3: MSE = 13798.82, MAE = 34.87
Fold 4: MSE = 13220.54, MAE = 26.76
Fold 5: MSE = 9736.78, MAE = 41.02

Mean MSE: 8501.26
Mean MAE: 29.92


In [10]:
# Pipeline B = preproc2 + baseline
# Define Pipeline B: preproc2 + Decision Tree Regressor (Baseline)
pipeline_b_dtr = Pipeline(steps=[
    ('preprocessing', preproc2),  # Preprocessing pipeline with preproc2
    ('regressor', DecisionTreeRegressor(random_state=42))  # Decision Tree Regressor as baseline
])

# Evaluate Pipeline B using cross_validate
print("Evaluating Pipeline B (preproc2 + Decision Tree Regressor)...")
cv_results = cross_validate(
    pipeline_b_dtr,
    X,
    y,
    cv=5,
scoring=['neg_mean_squared_error', 'neg_mean_absolute_error']  # Focus on error metrics
)

# Display results for each fold
print("\nCross-Validation Results (Error Metrics):")
for i in range(5):
    print(f"Fold {i+1}: "
          f"MSE = {-cv_results['test_neg_mean_squared_error'][i]:.2f}, "
          f"MAE = {-cv_results['test_neg_mean_absolute_error'][i]:.2f}")

# Display mean metrics
mean_mse = -cv_results['test_neg_mean_squared_error'].mean()
mean_mae = -cv_results['test_neg_mean_absolute_error'].mean()

print(f"\nMean MSE: {mean_mse:.2f}")
print(f"Mean MAE: {mean_mae:.2f}")

Evaluating Pipeline B (preproc2 + Decision Tree Regressor)...

Cross-Validation Results (Error Metrics):
Fold 1: MSE = 3947.42, MAE = 29.40
Fold 2: MSE = 2457.09, MAE = 23.66
Fold 3: MSE = 14351.79, MAE = 36.36
Fold 4: MSE = 13214.72, MAE = 25.99
Fold 5: MSE = 9642.45, MAE = 40.85

Mean MSE: 8722.70
Mean MAE: 31.25


In [11]:
# Pipeline C = preproc1 + advanced model
# Define Pipeline C: preproc1 + Gradient Boosting Regressor (Advanced Model)
pipeline_c_gb = Pipeline(steps=[
    ('preprocessing', preproc1),  # Preprocessing pipeline with preproc1
    ('regressor', GradientBoostingRegressor(random_state=42))  # Gradient Boosting Regressor as advanced model
])

# Evaluate Pipeline C using cross_validate
print("Evaluating Pipeline C (preproc1 + Gradient Boosting Regressor)...")
cv_results = cross_validate(
    pipeline_c_gb,
    X,
    y,
    cv=5,
    scoring=['neg_mean_squared_error', 'neg_mean_absolute_error']  # Focus on error metrics
)

# Display results for each fold
print("\nCross-Validation Results (Error Metrics):")
for i in range(5):
    print(f"Fold {i+1}: "
          f"MSE = {-cv_results['test_neg_mean_squared_error'][i]:.2f}, "
          f"MAE = {-cv_results['test_neg_mean_absolute_error'][i]:.2f}")

# Display mean metrics
mean_mse_c = -cv_results['test_neg_mean_squared_error'].mean()
mean_mae_c = -cv_results['test_neg_mean_absolute_error'].mean()

print(f"\nMean MSE: {mean_mse_c:.2f}")
print(f"Mean MAE: {mean_mae_c:.2f}")


Evaluating Pipeline C (preproc1 + Gradient Boosting Regressor)...

Cross-Validation Results (Error Metrics):
Fold 1: MSE = 2433.41, MAE = 29.37
Fold 2: MSE = 855.86, MAE = 14.58
Fold 3: MSE = 13762.42, MAE = 35.97
Fold 4: MSE = 7568.71, MAE = 30.33
Fold 5: MSE = 6917.08, MAE = 29.30

Mean MSE: 6307.50
Mean MAE: 27.91


In [12]:
# Pipeline D = preproc2 + advanced model
# Define Pipeline D: preproc2 + Gradient Boosting Regressor (Advanced Model)
pipeline_d_gb = Pipeline(steps=[
    ('preprocessing', preproc2),  # Preprocessing pipeline with preproc2
    ('regressor', GradientBoostingRegressor(random_state=42))  # Gradient Boosting Regressor as advanced model
])

# Evaluate Pipeline D using cross_validate
print("Evaluating Pipeline D (preproc2 + Gradient Boosting Regressor)...")
cv_results = cross_validate(
    pipeline_d_gb,
    X,
    y,
    cv=5,
    scoring=['neg_mean_squared_error', 'neg_mean_absolute_error']  # Focus on error metrics
)

# Display results for each fold
print("\nCross-Validation Results (Error Metrics):")
for i in range(5):
    print(f"Fold {i+1}: "
          f"MSE = {-cv_results['test_neg_mean_squared_error'][i]:.2f}, "
          f"MAE = {-cv_results['test_neg_mean_absolute_error'][i]:.2f}")

# Display mean metrics
mean_mse_c = -cv_results['test_neg_mean_squared_error'].mean()
mean_mae_c = -cv_results['test_neg_mean_absolute_error'].mean()

print(f"\nMean MSE: {mean_mse_c:.2f}")
print(f"Mean MAE: {mean_mae_c:.2f}")


Evaluating Pipeline D (preproc2 + Gradient Boosting Regressor)...

Cross-Validation Results (Error Metrics):
Fold 1: MSE = 2599.68, MAE = 29.26
Fold 2: MSE = 1016.01, MAE = 14.78
Fold 3: MSE = 13756.25, MAE = 35.91
Fold 4: MSE = 7311.66, MAE = 29.67
Fold 5: MSE = 7012.92, MAE = 30.56

Mean MSE: 6339.30
Mean MAE: 28.03


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [13]:
# Hyperparameter tuning for Pipeline A = preproc1 + Decision Tree Regressor (Baseline)

# Define parameter grid for Decision Tree Regressor
param_grid_dtr = {
    'regressor__max_depth': [5, 10, 20, None],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 5]
}

# Perform GridSearchCV
grid_search_dtr = GridSearchCV(
    estimator=pipeline_a_dtr,
    param_grid=param_grid_dtr,
    scoring='r2',  # Use R² as the evaluation metric
    cv=5,
    n_jobs=-1
)

grid_search_dtr.fit(X, y)

# Display best parameters and score
print("Best Parameters for Decision Tree Regressor:", grid_search_dtr.best_params_)
print("Best R² Score from GridSearchCV:", grid_search_dtr.best_score_)



Best Parameters for Decision Tree Regressor: {'regressor__max_depth': 5, 'regressor__min_samples_leaf': 5, 'regressor__min_samples_split': 2}
Best R² Score from GridSearchCV: -1.4786507675534384


In [14]:
# Hyperparameter tuning for Pipeline B = preproc2 + Decision Tree Regressor (Baseline)

# Define the hyperparameter grid for Decision Tree Regressor

param_grid_dtr = {
    'regressor__max_depth': [5, 10, 15, None],  # Depth of the tree
    'regressor__min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'regressor__min_samples_leaf': [1, 2, 5]    # Minimum samples in a leaf node
}

# Perform GridSearchCV on Pipeline B
grid_search_dtr = GridSearchCV(
    estimator=pipeline_b_dtr,
    param_grid=param_grid_dtr,
    scoring='r2',  # Use R² as the evaluation metric
    cv=5,          # 5-fold cross-validation
    n_jobs=-1      # Use all available processors for faster computation
)

# Fit the GridSearchCV to find the best parameters
grid_search_dtr.fit(X, y)

# Retrieve best parameters and best score
best_params_dtr = grid_search_dtr.best_params_
best_score_dtr = grid_search_dtr.best_score_

# Display the best parameters and corresponding R² score
print("Best Parameters for Decision Tree Regressor:", best_params_dtr)
print(f"Best R² Score from GridSearchCV: {best_score_dtr:.2f}")


Best Parameters for Decision Tree Regressor: {'regressor__max_depth': 5, 'regressor__min_samples_leaf': 5, 'regressor__min_samples_split': 2}
Best R² Score from GridSearchCV: -1.48


In [15]:
# Hyperparameter tuning for Pipeline C = preproc2 + Gradient Boosting Regressor (Advanced)


# Define parameter grid for Gradient Boosting Regressor
param_grid_gb = {
    'regressor__n_estimators': [100, 500, 1000],
    'regressor__learning_rate': [0.001, 0.01, 0.05, 0.1],
    'regressor__max_depth': [2, 3, 5, 7],
    'regressor__subsample': [0.6, 0.8, 1.0],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 5]
}

# Perform GridSearchCV on Pipeline C with Gradient Boosting
grid_search_gb = GridSearchCV(
    estimator=pipeline_c_gb,
    param_grid=param_grid_gb,
    scoring='r2',  # Use R² as the evaluation metric
    cv=5,          # 5-fold cross-validation
    n_jobs=-1      # Use all available processors for faster computation
)

# Fit GridSearchCV
print("Performing Grid Search for Gradient Boosting...")
grid_search_gb.fit(X, y)

# Retrieve best parameters and best score
best_params_gb = grid_search_gb.best_params_
best_score_gb = grid_search_gb.best_score_

# Display the best parameters and corresponding R² score
print("\nBest Parameters for Gradient Boosting Regressor:", best_params_gb)
print(f"Best Mean R² from GridSearchCV: {best_score_gb:.2f}")


Performing Grid Search for Gradient Boosting...

Best Parameters for Gradient Boosting Regressor: {'regressor__learning_rate': 0.001, 'regressor__max_depth': 2, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 500, 'regressor__subsample': 1.0}
Best Mean R² from GridSearchCV: -1.16


In [16]:
# Hyperparameter tuning for Pipeline D = preproc2 + Gradient Boosting Regressor (Advanced)

# Define parameter grid for Gradient Boosting Regressor
param_grid_gb = {
    'regressor__n_estimators': [100, 200, 300],  # Number of boosting stages
    'regressor__learning_rate': [0.1, 0.2],  # Learning rate for each stage
    'regressor__max_depth': [3, 5, 7],  # Maximum depth of the individual estimators
    'regressor__subsample': [0.8, 1.0], # Fraction of samples used for fitting
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 5]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search_gb = GridSearchCV(
    estimator=pipeline_d_gb,
    param_grid=param_grid_gb,
    scoring='r2',  # Use R² as the evaluation metric
    cv=5,          # 5-fold cross-validation
    n_jobs=-1      # Use all available processors for faster computation
)

# Fit GridSearchCV
print("Performing Grid Search for Gradient Boosting Regressor...")
grid_search_gb.fit(X, y)

# Retrieve the best parameters and the best R² score
best_params_gb = grid_search_gb.best_params_
best_score_gb = grid_search_gb.best_score_

# Display the best parameters and corresponding R² score
print("\nBest Parameters for Gradient Boosting Regressor:", best_params_gb)
print(f"Best Mean R² from GridSearchCV: {best_score_gb:.2f}")


Performing Grid Search for Gradient Boosting Regressor...

Best Parameters for Gradient Boosting Regressor: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 3, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 100, 'regressor__subsample': 1.0}
Best Mean R² from GridSearchCV: -4.30


# Evaluate

+ Which model has the best performance?

Gradient Boosting Regressor (Model 1) performed the best, achieving the highest (least negative) R² score of -1.16.
The Decision Tree Regressor performed consistently, but its R² score of -1.48 was worse than Gradient Boosting Regressor (Model 1).
The second Gradient Boosting model (Model 2) performed the worst, with a significantly lower R² score of -4.30.
The key difference between the two Gradient Boosting models is the learning rate and number of estimators:
Model 1 had learning_rate = 0.001 and 500 estimators.
Model 2 had learning_rate = 0.1 and 100 estimators.
The poor performance of Model 2 suggests that a higher learning rate and fewer estimators did not generalize well.
Answering the Question:
If the question is: Which model performed best and why?

The best-performing model is Gradient Boosting Regressor (Model 1) with an R² score of -1.16.
This is likely because it uses a lower learning rate (0.001) and more estimators (500), which allows for a more gradual learning process, reducing the risk of overfitting and improving performance.
The Decision Tree models performed similarly, but not as well as the best Gradient Boosting model.
The worst model was Gradient Boosting (Model 2) with an R² of -4.30, likely due to the higher learning rate (0.1) and fewer estimators (100), which might have caused the model to converge too quickly and underfit the data.

# Export

+ Save the best performing model to a pickle file.

In [ ]:
import pickle

# Assuming 'best_model' is the trained Gradient Boosting model
best_model = grid_search_gb.best_estimator_  # Ensure this refers to the best model from GridSearchCV

# Save the model to a file
with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Best model saved successfully as best_model.pkl!")

Best model saved successfully as best_model.pkl!


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train set size:", X_train.shape)
print("Test set size:", X_test.shape)

Train set size: (413, 12)
Test set size: (104, 12)


In [23]:
# Load the saved model
with open("best_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Make predictions
predictions = loaded_model.predict(X_test)  # Replace X_test with your test dataset


In [30]:
import pandas as pd

# Ensure X_test is a Pandas DataFrame
X_test = pd.DataFrame(X_test)

# Select a single observation
observation = X_test.iloc[[0]]  # Ensure this is a DataFrame (not Series)


In [31]:

# Compute SHAP values for this observation
shap_values = explainer(observation)

# Visualize feature contributions for this specific prediction
shap.waterfall_plot(shap_values[0])


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.